# My turn

In [30]:
import numpy as np

In [31]:
pb_data = np.loadtxt('mass_atten_pb.dat', delimiter=' ')
pb_energy = pb_data[:, 0]
pb_scatter = pb_data[:, 1]
pb_absorb = pb_data[:, 2]
pb_pair_production = pb_data[:, 3]
pb_total_XS = pb_scatter + pb_absorb + pb_pair_production
pb_E_depo = pb_data[:, 6]

In [32]:
def interpolate2(E, arr_E: np.ndarray, arr_inter: np.ndarray):
    return np.exp(np.interp(
    np.log(E),
    np.log(arr_E),
    np.log(arr_inter)
        ))

In [33]:
# My particle gambling functions
def r_sample(radius):
    r = radius* ((np.random.random())**(1/3))
    return r
def alpha_sample():
    return 2*np.pi*np.random.random()
def omega_sample():
    return 2*np.random.random() - 1

def mu_sample():
    return 2*np.random.random() - 1
def gamma_sample():
    return 2*np.pi*np.random.random()
    
def x_sample(radius, alpha, omega):
    return radius*np.cos(alpha)*np.sqrt(1 - (omega**2))
def y_sample(radius, alpha, omega):
    return radius*np.sin(alpha)*np.sqrt(1 - (omega**2))
def z_sample(radius, omega):
    return radius*omega
    

In [34]:
# Alright enough lollie-gagging (I think that's how you spell it). Time to code my MC-Particle Simulation
#rng = np.random.random()

class MyParticle:
    energy = 0
    d_energy = 0
    
    x = 0
    y = 0
    z = 0
    
    omega_x = 0
    omega_y = 0
    omega_z = 0
    
    alive = False  
    
    def __init__(self, x, y, z, energy, d_energy): 
        self.energy = energy
        self.d_energy = d_energy
        
        self.x = x
        self.y = y
        self.z = z
        
        mu = mu_sample()
        gamma = gamma_sample()
        
        self.omega_x = np.sqrt(1 - mu**2)*np.cos(gamma)
        self.omega_y = np.sqrt(1 - mu**2)*np.sin(gamma)
        self.omega_z = mu
        
        self.alive = True
    
    def print_vals(self):
        print('Here are the values of this particle')
        print(f"The energy is: {self.energy}")
        print(f"The x position is: {self.x}")
        print(f"The y position is: {self.y}")
        print(f"The z position is: {self.z}") 
    
    def make_dead(self):
        self.alive = False


In [35]:
class Sphere:
    radius = 0
    E_arr = 0
    E_t = 0
    
    def __init__(self, radius, E_arr:np.ndarray, E_t:np.ndarray):
        self.radius = radius
        self.E_arr = E_arr
        self.E_t = E_t
        
    
    def sample_particle(self, energy, d_energy):
        r = r_sample(self.radius)
        w = 2*np.random.random() - 1
        a = 2*np.pi*np.random.random()
        
        x = x_sample(r, a, w)
        y = y_sample(r, a, w)
        z = z_sample(r, w)
        
        return MyParticle(x, y, z, energy, d_energy) 
    
    # Distance determination functions -----------------

    # Distance to sphere surface
    
    def surf_dist(self, particle:MyParticle):
        b = 2*(particle.x*particle.omega_x + particle.y*particle.omega_y + particle.z*particle.omega_z)
        c = particle.x**2 + particle.y**2 + particle.z**2 - self.radius**2
        
        pos = (-b + np.sqrt(b**2 - 4*c))*0.5
        if pos > 0:
            return pos
            
        else:
            return (-b - np.sqrt(b**2 - 4*c))*0.5
    # Distance to next collision assuming
    def coll_dist_abs(self, particle:MyParticle):
        E_t = interpolate2(particle.energy, self.E_arr, self.E_t) 
        E_t_d = interpolate2(particle.energy + particle.d_energy, self.E_arr, self.E_t)
         
        d = self.surf_dist(particle)
        weight = np.exp(-(E_t_d - E_t) * d)
        #weight = E_t_d
        return -np.log(np.random.random())/E_t, weight, E_t, E_t_d
    
    def is_in_sphere(self, particle:MyParticle, x = 0, y = 0, z = 0):
        if (x - particle.x)**2 + (y - particle.y)**2 + (z - particle.z)**2 <= self.radius**2:
            return True
        else:
            return False
    
    

In [36]:
def history(histories, energy, dE, r, E_arr:np.ndarray ,E_t:np.ndarray):
    leaked = 0
    dleaked = 0
    sphere = Sphere(r, E_arr, E_t)
    E_t_holder = 0
    E_d_t_holder = 0
    
    particle_bank = np.array([sphere.sample_particle(energy, dE) for _ in range(int(histories))], dtype=object)
    bank_tracker = histories
    i = 0
    
    while i < bank_tracker:
        #E_t_current = interpolate2(particle_bank[i].energy, E_arr, E_t)
        
        while particle_bank[i].alive == True:
            l_s = sphere.surf_dist(particle_bank[i])
            l_c, w, E_t_holder, E_d_t_holder = sphere.coll_dist_abs(particle_bank[i])
            
            if l_s < l_c:
                particle_bank[i].make_dead()
                leaked += 1
                dleaked += 1*w
            else:
                particle_bank[i].make_dead()
        
        i += 1
        
    
    return leaked, dleaked, E_t_holder, E_d_t_holder



In [41]:
# Lets generate a monoenergetic sample of particles
# I am thinking 1E6 neutrons with an energy of 1 MeV scattered about a sphere of radius 5 units
mono_energy = 4.5 # MeV
dE = 45.5 # MeV
sphere_radius = 5
histories = 1E7
p_pb = 11.35 # g/cm3
E_t = pb_total_XS*p_pb

leaked, dleaked, E_t_holder, E_t_d_holder = history(histories, mono_energy, dE, sphere_radius, pb_energy, E_t)

print(f"In the purely absorbing sphere of radius {sphere_radius}, \n {leaked} particles were leaked out of {histories}")
print(f"Leakage rate for an energy of {mono_energy} is {leaked/histories}")
print(f"Leakage rate for an energy of {mono_energy + dE} is {dleaked/histories}")

In the purely absorbing sphere of radius 5, 
 2869870 particles were leaked out of 10000000.0
Leakage rate for an energy of 4.5 is 0.286987
Leakage rate for an energy of 50.0 is 0.1600989743045132


In [42]:
# Check
E_a = E_t_holder
P_0 = (0.375)*((E_a*sphere_radius)**(-3)) *(2*(E_a*sphere_radius)**2 - 1 + (1+2*E_a*sphere_radius)*np.exp(-2*E_a*sphere_radius))
print(f"E_t of {E_t_holder} should give {P_0}")

E_t of 0.4790677033304693 should give 0.2871352174466087


In [43]:
# Check
E_a = E_t_d_holder
#E_a = 0.57745395
P_0 = (0.375)*((E_a*sphere_radius)**(-3)) *(2*(E_a*sphere_radius)**2 - 1 + (1+2*E_a*sphere_radius)*np.exp(-2*E_a*sphere_radius))
print(f"E_t_d of {E_t_d_holder} should give {P_0}")

E_t_d of 0.9142764364999999 should give 0.16014298170376948
